In [52]:
from glob import glob
import os
from collections import defaultdict
from pathlib import Path
from enum import Enum
import json
import pandas as pd
import seaborn as sns

In [53]:
UNATE_SYNTHESIS_PATH = '../experiments/synthesis/unates/'
DEPS_SYNTHESIS_PATH = '../experiments/synthesis/dependency/'
UNATE_AND_DEPS_SYNTHESIS_PATH = '../experiments/synthesis/unates_and_dependency/'
PLAIN_SYNTHESIS_PATH = '../experiments/synthesis/plain/'

In [54]:
class Status(Enum):
    ERROR = "Error"
    SUCCESS = "Success"
    NOT_FOUND = "Not Found"
    TIMEOUT = "Timeout"
    
    def __repr__(self):
        return str(self.value)
    
    def __str__(self):
        return str(self.value)

In [55]:
def get_all_benchmarks(path):
    hoa_files = glob(os.path.join(path, "*.hoa"))
    names = [
        Path(f).stem
        for f in hoa_files
    ]
    return names

print("Total benchmark with Unates Synthesis" , len(get_all_benchmarks(UNATE_SYNTHESIS_PATH)))
print("Total benchmark with Deps Synthesis" , len(get_all_benchmarks(DEPS_SYNTHESIS_PATH)))
print("Total benchmark with Unates & Deps Synthesis" , len(get_all_benchmarks(UNATE_AND_DEPS_SYNTHESIS_PATH)))
print("Total benchmark with Plain Synthesis" , len(get_all_benchmarks(PLAIN_SYNTHESIS_PATH)))

Total benchmark with Unates Synthesis 207
Total benchmark with Deps Synthesis 207
Total benchmark with Unates & Deps Synthesis 207
Total benchmark with Plain Synthesis 207


In [56]:
def extract_vars_from_str(x):
    if len(x) == 0:
        return []
    return x.split(",")

In [57]:
def benchmark_loader(path: str, name: str):
    base = {
        'Name': name,
    }
    
    json_path = os.path.join(path, name + ".json")
    
    if not os.path.exists(json_path):
        return {
            **base,
            'Status': Status.NOT_FOUND
        }
    
    with open(json_path, 'r') as f:
        content = f.readlines()
        try:
            benchmark_json = json.loads(content[0])
        except Exception as e:
            return {
                **base,
                'Status': Status.ERROR,
                'Error': content
            }
    
    # Generic Benchmark Name
    output_vars = benchmark_json['output_vars']
    is_completed = benchmark_json['is_completed']
    total_duration = benchmark_json['total_time']
    
    if not is_completed:
        return {
            **base,
            'Status': Status.TIMEOUT,
            'Is Completed': is_completed,
            'Total Duration': total_duration
        }
    
    # Automaton Build
    is_automaton_build = benchmark_json['automaton']['is_built']
    automaton_build_duration = benchmark_json['automaton']['build_duration']
    total_states = benchmark_json['automaton']['prune_total_states']
    total_edges = benchmark_json['automaton']['total_edges']
    total_output_vars = len(benchmark_json['output_vars'])
    
    # Synthesis
    realizability = benchmark_json['synthesis']['independent_strategy']['realizability']
    indeps_synthesis_duration = benchmark_json['synthesis']['independent_strategy']['duration']
    deps_synthesis_duration = benchmark_json['synthesis']['dependent_strategy']['duration']
    
    # Unateness
    skipped_unate = benchmark_json['unate']['skipped_unate']
    if skipped_unate:
        total_unate_duration = None
        automaton_postprocessing_duration = None
        removed_states = None
        removed_edges = None
    else:
        total_unate_duration = benchmark_json['unate']['total_unate_duration']
        automaton_postprocessing_duration = benchmark_json['unate']['automaton_postprocess_duration']
        removed_states = total_states - benchmark_json['unate']['total_states_after_unate']
        removed_edges = total_edges - benchmark_json['unate']['total_edges_after_unate']

    vars_positive_unates = { var: 0 for var in output_vars }
    vars_negative_unates = { var: 0 for var in output_vars }
    vars_unknown_unates = { var: 0 for var in output_vars }
    state_positive_unates = { state: 0 for state in range(total_states) }
    state_negative_unates = { state: 0 for state in range(total_states) }
    states_not_unates = { state: 0 for state in range(total_states) }
    states_unknown_unates = { state: 0 for state in range(total_states) }
    states_complement_duration = { state: 0 for state in range(total_states) }
    states_complement_failed = { state: None for state in range(total_states) }
    states_removed_edges = { state: 0 for state in range(total_states) }
    states_impacted_edges = { state: 0 for state in range(total_states) }
    
    for state_unateness in benchmark_json['unate'].get('unate_states',[]):
        state = state_unateness['state']
        state_positive_vars = extract_vars_from_str(state_unateness['positive_unate_variables'])
        state_negative_vars = extract_vars_from_str(state_unateness['negative_unate_variables'])
        state_not_unates = extract_vars_from_str(state_unateness['not_unate_variables'])
        state_unknown_unates = extract_vars_from_str(state_unateness['unknown_unate_variables'])
        assert set(state_positive_vars).isdisjoint(set(state_negative_vars))
        
        states_removed_edges[state] = state_unateness['removed_edges']
        states_impacted_edges[state] = state_unateness['impacted_edges']
        
        states_complement_duration[state] = state_unateness['complement_duration']
        states_complement_failed[state] = not state_unateness['complement_succeeded']
        
        for var in state_positive_vars:
            vars_positive_unates[var] += 1
            state_positive_unates[state] += 1
        
        for var in state_negative_vars:
            vars_negative_unates[var] += 1
            state_negative_unates[state] += 1
            
        for var in state_unknown_unates:
            vars_unknown_unates[var] += 1
            states_unknown_unates[state] += 1
    
    # Dependency
    skipped_dependency = benchmark_json['dependencies']['skipped_dependencies']
    find_dependency_duration = benchmark_json['dependencies']['total_duration']
    total_dependent_vars = 0
    total_independent_vars = 0
    for tested_dependency in benchmark_json['tested_dependencies']:
        is_dependent = tested_dependency['is_dependent']
        if is_dependent:
            total_dependent_vars += 1
        else:
            total_independent_vars += 1
    
    return {
        # Group 1: General
        **base,
        'Status': Status.SUCCESS,
        'Is Completed': is_completed,
        'Total Duration': total_duration,
        'Realizability': realizability,
        'Applied Unate': not skipped_unate,
        'Applied Dependency': not skipped_dependency,
        'Independent Strategy Gates': benchmark_json['synthesis']['independent_strategy']['total_gates'],
        'Independent Synthesis Duration': indeps_synthesis_duration,
        'Dependent Synthesis Duration': deps_synthesis_duration,
        
        # Group 2: Automaton
        'Is Automaton Built': is_automaton_build,
        'Automaton Build Duration': automaton_build_duration,
        'Total Output Vars': total_output_vars,
        'Original Total States': total_states,
        'Original Total Edges': total_edges,
        
        # Unate Impactness
        'Total Unate Duration': total_unate_duration,
        'Automaton Postprocessing Duration': automaton_postprocessing_duration,
        'Total Impacted Edges': sum(states_impacted_edges.values()),
        'Total Removed Edges': sum(states_removed_edges.values()),

        # Group 3: Unates by States
        'States Failed by Complement': sum(1 for val in states_complement_failed.values() if val),
        'Total Complement Duration': sum(val for val in states_complement_duration.values()),
        'Total State with Positive Unates': sum(1 for val in state_positive_unates.values() if val > 0),
        'Total State with Negative Unates': sum(1 for val in state_negative_unates.values() if val > 0),
        'Total State with Unknown Unates': sum(1 for val in states_unknown_unates.values() if val > 0),
        
        # Group 4: Unates by Vars
        'Total Positive Unates Vars (At least 1)': sum(1 for val in vars_positive_unates.values() if val > 0),
        'Total Negative Unates Vars (At least 1)': sum(1 for val in vars_negative_unates.values() if val > 0),
        'Total Positive Unates Vars (All)': sum(1 for val in vars_positive_unates.values() if val == total_states),
        'Total Negative Unates Vars (All)': sum(1 for val in vars_negative_unates.values() if val == total_states),
        
        # Dependency
        'Total Dependent Variables': total_dependent_vars,
        'Total Independent Variables': total_independent_vars,
        'Dependency Ratio': total_dependent_vars / total_output_vars,
        'Find Dependency Duration': find_dependency_duration,
        'Find Dependency Completed': find_dependency_duration != -1,
    }

In [58]:
unates_synthesis_df = pd.DataFrame([
    benchmark_loader(UNATE_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(UNATE_SYNTHESIS_PATH)
])
unates_synthesis_df.head(5)
unates_synthesis_df.to_csv('./unates.csv', index=False)

In [78]:
benchmarks_with_unate = unates_synthesis_df[(unates_synthesis_df["Total Positive Unates Vars (At least 1)"] > 0) 
                    | (unates_synthesis_df["Total Negative Unates Vars (At least 1)"] > 0)]
benchmarks_with_unate[(benchmarks_with_unate["Realizability"] == "REALIZABLE") & (benchmarks_with_unate["Total Impacted Edges"] > 0)]

,Name,Status,Is Completed,Total Duration,Realizability,Applied Unate,Applied Dependency,Independent Strategy Gates,Independent Synthesis Duration,Dependent Synthesis Duration,...,Total Positive Unates Vars (At least 1),Total Negative Unates Vars (At least 1),Total Positive Unates Vars (All),Total Negative Unates Vars (All),Total Dependent Variables,Total Independent Variables,Dependency Ratio,Find Dependency Duration,Find Dependency Completed,Error
72,lilydemo08,Success,True,1.0,REALIZABLE,True,False,0.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
74,lilydemo20,Success,True,15.0,REALIZABLE,True,False,0.0,9.0,-1.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
91,lilydemo18,Success,True,37.0,REALIZABLE,True,False,10.0,20.0,-1.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
92,lilydemo24,Success,True,7.0,REALIZABLE,True,False,49.0,3.0,-1.0,...,4.0,2.0,2.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
118,lilydemo17,Success,True,30.0,REALIZABLE,True,False,5.0,3.0,-1.0,...,1.0,3.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
129,02,Success,True,67.0,REALIZABLE,True,False,115.0,47.0,-1.0,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
131,03,Success,True,2072.0,REALIZABLE,True,False,191.0,1937.0,-1.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
132,lilydemo14,Success,True,6.0,REALIZABLE,True,False,4.0,2.0,-1.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
137,lilydemo10,Success,True,3.0,REALIZABLE,True,False,0.0,1.0,-1.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,False,NaN
155,lilydemo13,Success,True,1.0,REALIZABLE,True,False,1.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,False,NaN


In [59]:
dependents_synthesis_df = pd.DataFrame([
    benchmark_loader(DEPS_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(DEPS_SYNTHESIS_PATH)
])
dependents_synthesis_df.head(5)
dependents_synthesis_df.to_csv('./dependents.csv', index=False)

In [60]:
plain_synthesis_df = pd.DataFrame([
    benchmark_loader(PLAIN_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(PLAIN_SYNTHESIS_PATH)
])
plain_synthesis_df.head(5)
plain_synthesis_df.to_csv('./plain.csv', index=False)

In [61]:
unates_and_deps_synthesis_df = pd.DataFrame([
    benchmark_loader(UNATE_AND_DEPS_SYNTHESIS_PATH, name)
    for name in get_all_benchmarks(UNATE_AND_DEPS_SYNTHESIS_PATH)
])
unates_and_deps_synthesis_df.head(5)
unates_and_deps_synthesis_df.to_csv('./unates_and_deps.csv', index=False)

# Sanity Checks

In [65]:
unates_synthesis_df[unates_synthesis_df["Total Removed Edges"] > 0]

,Name,Status,Is Completed,Total Duration,Realizability,Applied Unate,Applied Dependency,Independent Strategy Gates,Independent Synthesis Duration,Dependent Synthesis Duration,...,Total Positive Unates Vars (At least 1),Total Negative Unates Vars (At least 1),Total Positive Unates Vars (All),Total Negative Unates Vars (All),Total Dependent Variables,Total Independent Variables,Dependency Ratio,Find Dependency Duration,Find Dependency Completed,Error


## Automaton Build Duration

In [62]:
automaton_build_duration_df = pd.merge(
    pd.merge(
        plain_synthesis_df[["Name", "Automaton Build Duration"]],
        unates_and_deps_synthesis_df[["Name", "Automaton Build Duration"]],
        suffixes=('_plain', '_all'),
        on='Name'
    ),
    pd.merge(
        unates_synthesis_df[["Name", "Automaton Build Duration"]],
        dependents_synthesis_df[["Name", "Automaton Build Duration"]],
        suffixes=('_unates', '_deps'),
        on='Name'
    ),
    on='Name'
)
automaton_build_duration_df
# automaton_build_duration_df.to_csv('./automaton-build-duration.csv')

,Name,Automaton Build Duration_plain,Automaton Build Duration_all,Automaton Build Duration_unates,Automaton Build Duration_deps
0,TwoCountersInRangeA4,149.0,84.0,144.0,103.0
1,ltl2dba05,1.0,1.0,1.0,1.0
2,ltl2dba11,0.0,0.0,0.0,0.0
3,TwoCountersDisButA6,6094.0,4613.0,8410.0,5357.0
4,TorcsSteeringSimple,2.0,2.0,2.0,1.0
...,...,...,...,...,...
202,OneCounterGuiA1,8.0,5.0,8.0,6.0
203,EscalatorSmart,3.0,4.0,4.0,3.0
204,OneCounterGuiA0,6.0,4.0,9.0,5.0
205,ltl2dba20,3.0,2.0,3.0,2.0


## No Realiability Contradication

In [63]:
realizability_df = pd.merge(
    pd.merge(
        plain_synthesis_df[["Name", "Original Total States", "Total Output Vars", "Realizability"]],
        unates_and_deps_synthesis_df[["Name", "Realizability"]],
        suffixes=('_plain', '_all'),
        on='Name'
    ),
    pd.merge(
        unates_synthesis_df[["Name", "Realizability", "Total Impacted Edges", ]],
        dependents_synthesis_df[["Name", "Realizability", "Total Dependent Variables"]],
        suffixes=('_unates', '_deps'),
        on='Name'
    ),
    on='Name'
)
realizability_df

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
0,TwoCountersInRangeA4,6.0,8.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,3.0
1,ltl2dba05,13.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,21.0,REALIZABLE,0.0
2,ltl2dba11,5.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,2.0,REALIZABLE,0.0
3,TwoCountersDisButA6,10.0,13.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,4.0
4,TorcsSteeringSimple,4.0,3.0,REALIZABLE,REALIZABLE,REALIZABLE,0.0,REALIZABLE,1.0
...,...,...,...,...,...,...,...,...,...
202,OneCounterGuiA1,7.0,9.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,3.0
203,EscalatorSmart,14.0,3.0,REALIZABLE,REALIZABLE,REALIZABLE,0.0,REALIZABLE,1.0
204,OneCounterGuiA0,6.0,9.0,UNREALIZABLE,UNREALIZABLE,UNREALIZABLE,0.0,UNREALIZABLE,3.0
205,ltl2dba20,19.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,12.0,REALIZABLE,0.0


In [ ]:
# impacted_by_unate = realizability_df[realizability_df['Total Impacted Edges'] > 0]

In [66]:
len(realizability_df[(realizability_df["Realizability_plain"] == "REALIZABLE")])

142

In [71]:
common_realizable = realizability_df[(realizability_df["Realizability_plain"] == "REALIZABLE") & (realizability_df["Realizability_unates"] == "REALIZABLE")]
len(common_realizable)

102

In [72]:
len(common_realizable[common_realizable["Total Impacted Edges"] > 0])

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
72,lilydemo08,4.0,1.0,REALIZABLE,REALIZABLE,REALIZABLE,4.0,REALIZABLE,0.0
74,lilydemo20,24.0,3.0,REALIZABLE,REALIZABLE,REALIZABLE,8.0,REALIZABLE,0.0
92,lilydemo24,16.0,4.0,REALIZABLE,NaN,REALIZABLE,78.0,REALIZABLE,0.0
118,lilydemo17,17.0,3.0,REALIZABLE,REALIZABLE,REALIZABLE,33.0,REALIZABLE,0.0
129,02,93.0,4.0,REALIZABLE,REALIZABLE,REALIZABLE,19.0,REALIZABLE,0.0
131,03,597.0,5.0,REALIZABLE,REALIZABLE,REALIZABLE,35.0,REALIZABLE,0.0
132,lilydemo14,13.0,2.0,REALIZABLE,REALIZABLE,REALIZABLE,29.0,REALIZABLE,0.0
137,lilydemo10,6.0,2.0,REALIZABLE,REALIZABLE,REALIZABLE,12.0,REALIZABLE,0.0
155,lilydemo13,4.0,1.0,REALIZABLE,REALIZABLE,REALIZABLE,1.0,REALIZABLE,0.0
162,05,10201.0,5.0,REALIZABLE,REALIZABLE,REALIZABLE,130.0,REALIZABLE,0.0


In [69]:
not_common_realizable = realizability_df[(realizability_df["Realizability_plain"] == "REALIZABLE") & (realizability_df["Realizability_unates"] != "REALIZABLE")]
not_common_realizable.head(5)

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
1,ltl2dba05,13.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,21.0,REALIZABLE,0.0
2,ltl2dba11,5.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,2.0,REALIZABLE,0.0
7,ltl2dba10,10.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,7.0,REALIZABLE,0.0
8,ltl2dba04,12.0,1.0,REALIZABLE,UNREALIZABLE,NaN,NaN,REALIZABLE,0.0
10,ltl2dba12,9.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,11.0,REALIZABLE,0.0


In [ ]:
different_realizability_df = realizability_df[
    (
        (realizability_df['Realizability_unates'] != realizability_df['Realizability_deps']) |
        (realizability_df['Realizability_unates'] != realizability_df['Realizability_plain']) | 
        (realizability_df['Realizability_unates'] != realizability_df['Realizability_all']) | 
        (realizability_df['Realizability_all'] != realizability_df['Realizability_plain']) | 
        (realizability_df['Realizability_all'] != realizability_df['Realizability_deps']) | 
        (realizability_df['Realizability_deps'] != realizability_df['Realizability_plain']) 
    )
    & (realizability_df['Realizability_unates'].notna())
    & (realizability_df['Realizability_deps'].notna())
]
different_realizability_df

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
1,ltl2dba05,13.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,21.0,REALIZABLE,0.0
2,ltl2dba11,5.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,2.0,REALIZABLE,0.0
7,ltl2dba10,10.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,7.0,REALIZABLE,0.0
10,ltl2dba12,9.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,11.0,REALIZABLE,0.0
11,ltl2dba06,15.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,19.0,REALIZABLE,0.0
15,amba_decomposed_tsingle,21.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,5.0,REALIZABLE,0.0
23,ltl2dba07,33.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,55.0,REALIZABLE,0.0
24,ltl2dba13,11.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,12.0,REALIZABLE,0.0
29,ltl2dba17,28.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,4.0,REALIZABLE,0.0
30,ltl2dba03,12.0,1.0,REALIZABLE,UNREALIZABLE,UNREALIZABLE,15.0,REALIZABLE,0.0


In [ ]:
different_realizability_df[""]

In [ ]:
different_realizability_df[different_realizability_df["Total Dependent Variables"] > 0]

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
37,SensorInit,8.0,17.0,REALIZABLE,NaN,REALIZABLE,0.0,REALIZABLE,9.0


In [ ]:
different_realizability_df[different_realizability_df["Total Impacted Edges"] == 0]

,Name,Original Total States,Total Output Vars,Realizability_plain,Realizability_all,Realizability_unates,Total Impacted Edges,Realizability_deps,Total Dependent Variables
37,SensorInit,8.0,17.0,REALIZABLE,NaN,REALIZABLE,0.0,REALIZABLE,9.0


## All DataFrame has same order

In [ ]:
def verify_name_order(*dataframes):
    for i in range(1, len(dataframes)):
        if not dataframes[i - 1]['Name'].equals(dataframes[i]['Name']):
            return False
    return True

In [ ]:
verify_name_order(
    unates_and_deps_synthesis_df,
    plain_synthesis_df,
    dependents_synthesis_df,
    unates_synthesis_df
)

True